In [3]:
from google.colab import drive
drive.mount('/content/drive/',force_remount=True)
data_path = "/content/drive/MyDrive/Colab Notebooks/DL/"  # this is your drive

Mounted at /content/drive/


In [11]:
import json

# 读取原始JSON文件
with open(data_path + 'test_data/lCD-Coding-A.json', 'r', encoding='utf-8') as f:
    data = json.load(f)

# 处理数据
processed_data = []
for item in data:
    if '预测结果' in item:
        # 以|分割预测结果
        split_result = item['预测结果'].split('|')
        main_code = split_result[0].replace('[', '') if len(split_result) > 0 else ""
        other_codes = split_result[1].replace(']', '') if len(split_result) > 1 else ""

        # 创建新的格式
        new_item = {
            "主要诊断编码": main_code,
            "其他诊断编码": other_codes
        }
        processed_data.append(new_item)

# 保存到新文件
with open(data_path + 'test_data/ICD-Coding-split.json', 'w', encoding='utf-8') as f:
    json.dump(processed_data, f, ensure_ascii=False, indent=4)

print("处理完成，已保存到ICD-Coding-split.json")

处理完成，已保存到ICD-Coding-split.json


In [12]:
# 读取两个json文件
def merge_json_files(file1_path, file2_path):
    # 读取第一个json文件
    with open(file1_path, 'r', encoding='utf-8') as f1:
        data1 = json.load(f1)

    # 读取第二个json文件
    with open(file2_path, 'r', encoding='utf-8') as f2:
        data2 = json.load(f2)

    # 检查两个列表长度是否相同
    if len(data1) != len(data2):
        raise ValueError("两个JSON文件的对象数量不匹配")

    # 合并对应的对象
    merged_data = []
    for item1, item2 in zip(data1, data2):
        merged_item = {**item1, **item2}  # 合并两个字典
        merged_data.append(merged_item)

    return merged_data

# 示例使用
try:
    # 假设文件名为 file1.json 和 file2.json
    result = merge_json_files(data_path + 'test_data/ICD-Coding-test-A.json', data_path + 'test_data/ICD-Coding-split.json')

    # 打印结果
    # print(json.dumps(result, ensure_ascii=False, indent=2))

    # 如果需要保存到新文件
    with open(data_path + 'test_data/ICD-Coding-test.json', 'w', encoding='utf-8') as f:
        json.dump(result, f, ensure_ascii=False, indent=2)

except Exception as e:
    print(f"发生错误: {e}")

In [13]:
import re

# 假设你的JSON文件路径
json_file_path = data_path + 'test_data/ICD-Coding-test.json'

# 读取JSON文件
with open(json_file_path, 'r', encoding='utf-8') as f:
    data = json.load(f)

# 存放最终结果
result = []

# 遍历每个病例
for record in data:
    #病例标识
    bingli = record.get('病案标识', '')

    #主要诊断编码和其他诊断编码
    main_code = record.get('主要诊断编码', '')
    other_codes = record.get('其他诊断编码', '')


    # 主要诊断字段组合
    zhusu = record.get('主诉', '')
    xianbingshi = record.get('现病史', '')
    ruyuanqingkuang = record.get('入院情况', '')

    # 入院诊断处理
    ruyuan_zhenduan = record.get('入院诊断', '')
    # 提取 "1." 到 "2." 之间的内容
    ryzd_1 = ''
    match_1 = re.search(r'1\.(.*?)(2\.|$)', ruyuan_zhenduan, re.S)
    if match_1:
        ryzd_1 = match_1.group(1).strip()

    # 诊疗经过和出院情况
    zhenliao_guocheng = record.get('诊疗经过', '')
    chuyuan_qingkuang = record.get('出院情况', '')

    # 合并主要诊断文本
    main_text = f"{zhusu} {xianbingshi} {ruyuanqingkuang} {ryzd_1} {zhenliao_guocheng} {chuyuan_qingkuang}".strip()

    # 其他诊断字段组合
    jiwangshi = record.get('既往史', '')
    geren_shi = record.get('个人史', '')
    hunyin_shi = record.get('婚姻史', '')
    jiazu_shi = record.get('家族史', '')

    # 提取入院诊断 "2." 后的部分
    ryzd_2plus = ''
    match_2 = re.search(r'2\.(.*)', ruyuan_zhenduan, re.S)
    if match_2:
        ryzd_2plus = match_2.group(1).strip()

    # 出院医嘱
    chuyuan_yizhu = record.get('出院医嘱', '')

    # 合并其他诊断文本
    other_text = f"{jiwangshi} {geren_shi} {hunyin_shi} {jiazu_shi} {ryzd_2plus} {zhenliao_guocheng} {chuyuan_yizhu}".strip()

    # 拼装结果
    result.append({
        '病案标识': bingli,
        '主要诊断病例文本': main_text,
        '其他诊断病例文本': other_text,
        '主要诊断编码': main_code,
        '其他诊断编码': other_codes

    })

# 打印或者保存结果
output_file_path = data_path + 'test_data/test_combine_text.json'
with open(output_file_path, 'w', encoding='utf-8') as f:
    json.dump(result, f, ensure_ascii=False, indent=4)

print(f"处理完成，结果已保存到 {output_file_path}")


处理完成，结果已保存到 /content/drive/MyDrive/Colab Notebooks/DL/test_data/test_combine_text.json


In [2]:
!pip install paddlepaddle-gpu==2.6.0.post120 -f https://www.paddlepaddle.org.cn/whl/linux/mkl/avx/stable.html
!pip install --upgrade paddlenlp

Looking in links: https://www.paddlepaddle.org.cn/whl/linux/mkl/avx/stable.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 788.0/788.0 MB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.5/65.5 kB 2.0 MB/s eta 0:00:00
  Attempting uninstall: opt-einsum
    Found existing installation: opt_einsum 3.4.0
    Uninstalling opt_einsum-3.4.0:
      Successfully uninstalled opt_einsum-3.4.0
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 3.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 41.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 487.4/487.4 kB 41.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.9/86.9 kB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 128.7/128.7 kB 14.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.0/16.0 MB 108.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 10.1 MB/s eta 0:0

In [6]:
import paddle
import time
from paddlenlp import Taskflow

print("PaddlePaddle版本：", paddle.__version__)
print("是否使用GPU：", paddle.device.is_compiled_with_cuda())
print("当前设备：", paddle.device.get_device())

PaddlePaddle版本： 2.6.0
是否使用GPU： True
当前设备： gpu:0


In [8]:
import json

# 定义Schema
entity_schema = [
    '疾病', '症状', '检查项目', '治疗方法',
    '身体部位', '发病时间', '药物', '手术'
]

relation_schema = {
    '疾病': ['相关症状', '检查方法', '治疗方法'],
    '症状': ['诱发因素', '缓解因素', '伴随症状'],
    '治疗方法': ['治疗疾病', '用药剂量']
}

# 联合实体+关系抽取Schema
full_schema = entity_schema + list(relation_schema.keys())

# 加载医疗模型（自动加速）
try:
    ie_medical = Taskflow("information_extraction",
                          schema=full_schema,
                          task_path="uie-medical-base",
                          device_id=0)
except:
    ie_medical = Taskflow("information_extraction",
                          schema=full_schema,
                          device_id=0)

# 去重函数
def deduplicate_entities_and_relations(results):
    deduped_results = []
    for item in results:
        deduped_item = {}
        for entity_type, entities in item.items():
            seen = set()
            deduped_entities = []
            for entity in entities:
                text = entity['text']
                if text not in seen:
                    seen.add(text)
                    if 'relations' in entity:
                        relations = {}
                        for rel_type, rel_list in entity['relations'].items():
                            rel_seen = set()
                            dedup_rel_list = []
                            for rel in rel_list:
                                rel_text = rel['text']
                                if rel_text not in rel_seen:
                                    rel_seen.add(rel_text)
                                    dedup_rel_list.append(rel)
                            relations[rel_type] = dedup_rel_list
                        entity['relations'] = relations
                    deduped_entities.append(entity)
            deduped_item[entity_type] = deduped_entities
        deduped_results.append(deduped_item)
    return deduped_results


def simplify_entity_text(data):
    simplified_data = []
    for item in data:
        simplified_item = {}
        for entity_type, entities in item.items():
            # 只提取 text 字段，拼接成字符串
            texts = [entity['text'] for entity in entities]
            simplified_item[entity_type] = '，'.join(texts)
        simplified_data.append(simplified_item)
    return simplified_data

# 开始计时（整体运行时间）
start_time = time.time()

# 加载result JSON文件（假设文件叫 combine_text.json）
print("⏳ 开始加载 JSON 文件...")
load_start = time.time()
with open(data_path + 'test_data/test_combine_text.json', 'r', encoding='utf-8') as f:
    result_data = json.load(f)
load_end = time.time()
print(f"✅ JSON 文件加载完成，耗时: {load_end - load_start:.2f} 秒")

# 存储最终结果
final_output = []

# 遍历处理每条记录
print(f"\n📊 开始处理 {len(result_data)} 条记录...")
for idx, item in enumerate(result_data):
    loop_start = time.time()

    # 病例标识
    bingli = item.get('病案标识', '')

    # 主要诊断编码和其他诊断编码
    main_code = item.get('主要诊断编码', '')
    other_codes = item.get('其他诊断编码', '')

    main_text = item.get("主要诊断病例文本", "")
    other_text = item.get("其他诊断病例文本", "")

    # 对主要诊断文本执行信息抽取
    print(f"\n🚀 正在抽取第 {idx+1} 条记录的主要诊断病例文本...")
    main_extract_start = time.time()
    main_results = ie_medical(main_text)
    deduped_main = deduplicate_entities_and_relations(main_results)
    simplified_main = simplify_entity_text(deduped_main)
    main_extract_end = time.time()
    print(f"✅ 主要诊断抽取完成，耗时: {main_extract_end - main_extract_start:.2f} 秒")

    # 对其他诊断文本执行信息抽取
    print(f"\n🚀 正在抽取第 {idx+1} 条记录的其他诊断病例文本...")
    other_extract_start = time.time()
    other_results = ie_medical(other_text)
    deduped_other = deduplicate_entities_and_relations(other_results)
    simplified_other = simplify_entity_text(deduped_other)
    other_extract_end = time.time()
    print(f"✅ 其他诊断抽取完成，耗时: {other_extract_end - other_extract_start:.2f} 秒")

    # 组合结果
    final_output.append({
        '病案标识': bingli,
        "抽取后主要诊断病例文本": simplified_main,
        "抽取后其他诊断病例文本": simplified_other,
        '主要诊断编码': main_code,
        '其他诊断编码': other_codes
    })

    loop_end = time.time()
    print(f"📌 第 {idx+1}/{len(result_data)} 条记录处理完成，总耗时: {loop_end - loop_start:.2f} 秒")

# 保存结果到文件
print("\n⏳ 开始保存结果到文件...")
save_start = time.time()
output_file = data_path + "test_data/test_extracted_text.json"
with open(output_file, 'w', encoding='utf-8') as f:
    json.dump(final_output, f, ensure_ascii=False, indent=4)
save_end = time.time()
print(f"✅ 结果保存完成，耗时: {save_end - save_start:.2f} 秒")

# 结束计时（整体运行时间）
end_time = time.time()
total_time = end_time - start_time
print(f"\n🎉 全部处理完成，总耗时: {total_time:.2f} 秒")
print(f"结果已保存到：{output_file}")

[2025-03-17 09:06:04,117] [    INFO] - We are using <class 'paddlenlp.transformers.ernie.tokenizer.ErnieTokenizer'> to load 'uie-medical-base'.


⏳ 开始加载 JSON 文件...
✅ JSON 文件加载完成，耗时: 0.29 秒

📊 开始处理 200 条记录...

🚀 正在抽取第 1 条记录的主要诊断病例文本...


/usr/local/lib/python3.11/dist-packages/paddlenlp/transformers/tokenizer_utils_base.py:2512: FutureWarning: The `max_seq_len` argument is deprecated and will be removed in a future version, please use `max_length` instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/paddlenlp/transformers/tokenizer_utils_base.py:1912: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


✅ 主要诊断抽取完成，耗时: 1.40 秒

🚀 正在抽取第 1 条记录的其他诊断病例文本...
✅ 其他诊断抽取完成，耗时: 0.53 秒
📌 第 1/200 条记录处理完成，总耗时: 1.94 秒

🚀 正在抽取第 2 条记录的主要诊断病例文本...
✅ 主要诊断抽取完成，耗时: 0.42 秒

🚀 正在抽取第 2 条记录的其他诊断病例文本...
✅ 其他诊断抽取完成，耗时: 0.31 秒
📌 第 2/200 条记录处理完成，总耗时: 0.73 秒

🚀 正在抽取第 3 条记录的主要诊断病例文本...
✅ 主要诊断抽取完成，耗时: 0.55 秒

🚀 正在抽取第 3 条记录的其他诊断病例文本...
✅ 其他诊断抽取完成，耗时: 0.48 秒
📌 第 3/200 条记录处理完成，总耗时: 1.02 秒

🚀 正在抽取第 4 条记录的主要诊断病例文本...
✅ 主要诊断抽取完成，耗时: 0.40 秒

🚀 正在抽取第 4 条记录的其他诊断病例文本...
✅ 其他诊断抽取完成，耗时: 0.42 秒
📌 第 4/200 条记录处理完成，总耗时: 0.82 秒

🚀 正在抽取第 5 条记录的主要诊断病例文本...
✅ 主要诊断抽取完成，耗时: 0.26 秒

🚀 正在抽取第 5 条记录的其他诊断病例文本...
✅ 其他诊断抽取完成，耗时: 0.24 秒
📌 第 5/200 条记录处理完成，总耗时: 0.50 秒

🚀 正在抽取第 6 条记录的主要诊断病例文本...
✅ 主要诊断抽取完成，耗时: 0.47 秒

🚀 正在抽取第 6 条记录的其他诊断病例文本...
✅ 其他诊断抽取完成，耗时: 0.46 秒
📌 第 6/200 条记录处理完成，总耗时: 0.94 秒

🚀 正在抽取第 7 条记录的主要诊断病例文本...
✅ 主要诊断抽取完成，耗时: 0.43 秒

🚀 正在抽取第 7 条记录的其他诊断病例文本...
✅ 其他诊断抽取完成，耗时: 0.32 秒
📌 第 7/200 条记录处理完成，总耗时: 0.75 秒

🚀 正在抽取第 8 条记录的主要诊断病例文本...
✅ 主要诊断抽取完成，耗时: 0.55 秒

🚀 正在抽取第 8 条记录的其他诊断病例文本...
✅ 其他诊断抽取完成，耗时: 0.38 秒
📌 第 8/200 条记录处理完成，总耗时: 0.93 秒

🚀 

In [9]:
# 读取你的JSON文件路径
json_file = data_path + 'test_data/test_extracted_text.json'  # 修改为你自己的JSON文件路径

with open(json_file, 'r', encoding='utf-8') as f:
    data = json.load(f)

print(f"数据总量：{len(data)} 条")

def extract_text(diagnosis_list):
    """
    将单个诊断中的所有字段拼接为文本
    """
    text = []
    for diag in diagnosis_list:
        for key in ['疾病', '症状', '检查项目', '治疗方法', '身体部位', '药物', '手术']:
            value = diag.get(key, '')
            # 字符串格式直接拼接
            if isinstance(value, str):
                text.append(f"{key}：{value}")
            # 列表或其他格式，转为字符串
            elif isinstance(value, list):
                text.append(f"{key}：{'，'.join(value)}")
    return '；'.join(text)

dataset = []

for item in data:
    # 文本1和文本2
    text1_raw = item.get('抽取后主要诊断病例文本', [])
    text2_raw = item.get('抽取后其他诊断病例文本', [])

    text1 = extract_text(text1_raw)
    text2 = extract_text(text2_raw)

    # 标签1和标签2
    label1 = item.get('主要诊断编码', '')  # 单标签
    label2 = item.get('其他诊断编码', [])  # 多标签列表

    # 加入数据集
    dataset.append({
        'text1': text1,
        'text2': text2,
        'label1': label1,
        'label2': label2
    })

print(f"已处理数据条数：{len(dataset)}")


# 保存为JSON（如果需要）
with open(data_path + 'test_data/test_data.json', 'w', encoding='utf-8') as f:
    json.dump(dataset, f, ensure_ascii=False, indent=4)


print("训练集和验证集已保存完成！")

数据总量：200 条
已处理数据条数：200
训练集和验证集已保存完成！
